# Preparación de ambiente

## Librerías

In [1]:
import pandas as pd

# Carga de datos

In [2]:
sales = pd.read_csv('data/sales_train.csv')
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [3]:
items = pd.read_csv('data/items.csv')
items

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [4]:
categories = pd.read_csv('data/item_categories.csv')
categories

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [5]:
shops = pd.read_csv('data/shops.csv')
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


Queremos integrar toda la información en un único dataset, pero antes tenemos que hacer algunas validaciones. 
- Primero verificamos que los id's de los catálogos 'items', 'categories' y 'shops' sean únicos:

In [6]:
items['item_id'].value_counts().max(), categories['item_category_id'].value_counts().max(), shops['shop_id'].value_counts().max()

(np.int64(1), np.int64(1), np.int64(1))

- Ahora verificamos que a cada 'item_id' le corresponda un único 'item_category_id':

In [7]:
item_cat_check = items[['item_id', 'item_category_id']].drop_duplicates()
item_cat_check = item_cat_check.groupby('item_id').agg(
    n_categories = ('item_category_id', 'nunique'), 
    categories = ('item_category_id', lambda x: sorted(x.unique()))
    )
item_cat_check.reset_index(inplace = True)
item_cat_check

,item_id,n_categories,categories
0,0,1,[40]
1,1,1,[76]
2,2,1,[40]
3,3,1,[40]
4,4,1,[40]
...,...,...,...
22165,22165,1,[31]
22166,22166,1,[54]
22167,22167,1,[49]
22168,22168,1,[62]


In [8]:
item_cat_check['n_categories'].value_counts()

n_categories
1    22170
Name: count, dtype: int64

Hecho esto podemos unir los datasets:

In [9]:
data = pd.merge(sales, items, how = 'left', on = 'item_id')
data = pd.merge(data, categories, how = 'left', on = 'item_category_id')
data = pd.merge(data, shops, how = 'left', on = 'shop_id')
data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""
...,...,...,...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,V/A Nu Jazz Selection (digipack),55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935845,09.10.2015,33,25,7460,299.00,1.0,V/A The Golden Jazz Collection 1 2CD,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935846,14.10.2015,33,25,7459,349.00,1.0,V/A The Best Of The 3 Tenors,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935847,22.10.2015,33,25,7440,299.00,1.0,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Музыка - MP3,"Москва ТРК ""Атриум"""


# EDA

## Data cleaning

Agrupamos los tipos de variable y revisamos que tengan el tipo de dato correcto:

In [10]:
ids = ['date', 'shop_id', 'item_id', 'item_category_id']
cat = ['item_name', 'item_category_name', 'shop_name']
num = ['date_block_num', 'item_price', 'item_cnt_day']

In [11]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                str    
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           str    
 7   item_category_id    int64  
 8   item_category_name  str    
 9   shop_name           str    
dtypes: float64(2), int64(4), str(4)
memory usage: 224.0 MB


Cambiamos las fechas al tipo correcto:

In [12]:
data['date'] = pd.to_datetime(data['date'], format = "%d.%m.%Y")

Revisamos si hay registros duplicados:

In [13]:
data[data.duplicated(keep = False)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
76961,2013-01-05,0,54,20130,149.0,1.0,УЧЕНИК ЧАРОДЕЯ (регион),40,Кино - DVD,"Химки ТЦ ""Мега"""
76962,2013-01-05,0,54,20130,149.0,1.0,УЧЕНИК ЧАРОДЕЯ (регион),40,Кино - DVD,"Химки ТЦ ""Мега"""
1435365,2014-02-23,13,50,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Тюмень ТЦ ""Гудвин"""
1435367,2014-02-23,13,50,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Тюмень ТЦ ""Гудвин"""
1496765,2014-03-23,14,21,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Москва МТРЦ ""Афи Молл"""
1496766,2014-03-23,14,21,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Москва МТРЦ ""Афи Молл"""
1671872,2014-05-01,16,50,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Тюмень ТЦ ""Гудвин"""
1671873,2014-05-01,16,50,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Тюмень ТЦ ""Гудвин"""
1866322,2014-07-12,18,25,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Москва ТРК ""Атриум"""
1866340,2014-07-12,18,25,3423,999.0,1.0,"Far Cry 3 (Classics) [Xbox 360, русская версия]",23,Игры - XBOX 360,"Москва ТРК ""Атриум"""


Sí los hay, procedemos a eliminarlos:

In [14]:
data.drop_duplicates(inplace = True)
data.reset_index(drop = True, inplace = True)
data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-01-02,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""
...,...,...,...,...,...,...,...,...,...,...
2935838,2015-10-10,33,25,7409,299.00,1.0,V/A Nu Jazz Selection (digipack),55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935839,2015-10-09,33,25,7460,299.00,1.0,V/A The Golden Jazz Collection 1 2CD,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935840,2015-10-14,33,25,7459,349.00,1.0,V/A The Best Of The 3 Tenors,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2935841,2015-10-22,33,25,7440,299.00,1.0,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Музыка - MP3,"Москва ТРК ""Атриум"""


Ahora hacemos una revisión general por tipo de variable comenzando con la fecha, id's y numéricas:

In [15]:
data[ids + num].describe()

,date,shop_id,item_id,item_category_id,date_block_num,item_price,item_cnt_day
count,2935843,2.935843e+06,2.935843e+06,2.935843e+06,2.935843e+06,2.935843e+06,2.935843e+06
mean,2014-04-03 05:44:38.044159,3.300171e+01,1.019723e+04,4.000141e+01,1.456991e+01,8.908535e+02,1.242641e+00
min,2013-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,2013-08-01 00:00:00,2.200000e+01,4.476000e+03,2.800000e+01,7.000000e+00,2.490000e+02,1.000000e+00
50%,2014-03-04 00:00:00,3.100000e+01,9.343000e+03,4.000000e+01,1.400000e+01,3.990000e+02,1.000000e+00
75%,2014-12-05 00:00:00,4.700000e+01,1.568400e+04,5.500000e+01,2.300000e+01,9.990000e+02,1.000000e+00
max,2015-10-31 00:00:00,5.900000e+01,2.216900e+04,8.300000e+01,3.300000e+01,3.079800e+05,2.169000e+03
std,NaN,1.622698e+01,6.324293e+03,1.710076e+01,9.422992e+00,1.729801e+03,2.618837e+00


In [16]:
data[ids + num].isna().sum()

date                0
shop_id             0
item_id             0
item_category_id    0
date_block_num      0
item_price          0
item_cnt_day        0
dtype: int64

In [17]:
data[ids + num].isnull().sum()

date                0
shop_id             0
item_id             0
item_category_id    0
date_block_num      0
item_price          0
item_cnt_day        0
dtype: int64

No encontramos valores faltantes. Las fechas se encuentran en el rango esperado. Las variables identificadoras tienen valores razonables. Sólo queda profundizar en las variables 'item_price' e 'item_cnt_day' que tienen valores negativos.

Ahora continuamos con las variables categóricas:

In [18]:
for c in cat:
    print(data[c].value_counts(), '\n\n')

item_name
Фирменный пакет майка 1С Интерес белый (34*42) 45 мкм                                    31340
Playstation Store пополнение бумажника: Карта оплаты 1000 руб.                            9408
Прием денежных средств для 1С-Онлайн                                                      9067
Diablo III [PC, Jewel, русская версия]                                                    7479
Kaspersky Internet Security Multi-Device Russian Edition. 2-Device 1 year Renewal Box     6853
                                                                                         ...  
АХ, ВОДЕВИЛЬ, ВОДЕВИЛЬ (rem)                                                                 1
XBOX 360 S 250GB CONSOLE/KINECT BUNDLE KUNG FU PAL EN/RU RUSSIA                              1
WARHAMMER ACCESSORIES: Khorne Dice арт. 65-13                                                1
WARHAMMER MINIATURES: Dark Elf Doomfire Warlocks арт. 85-14                                  1
WARHAMMER MINIATURES: Stormcast Eternals

Podemos ver que las variables se encuentran muy pulverizadas, pero al parecer hay elementos dentro de cada una de ellas que pueden extraerse para generar grupos más grandes. Se explorará esta opción en la sección de Feature Engineering.

# Feature Engineering

In [20]:
data.to_feather('data/clean_data.feather')

# Model

## Training

## Evaluation

# Prediction